<a href="https://colab.research.google.com/github/Damilola-ops/Text-de-shuffling/blob/main/Text_de_shuffling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import EncoderDecoderModel, EncoderDecoderConfig, BertTokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertConfig
import torch
import pandas as pd 
import numpy as np
import sklearn.preprocessing 
from torch.nn import functional as F 
import datasets
from datasets import load_dataset


In [ ]:
train_df = pd.read_csv(r"C:\Users\USER\Datasets\Text-Deshuffling\train (8).csv")
train_df = train_df.dropna()
test_df = pd.read_csv(r"C:\Users\USER\Datasets\Text-Deshuffling\test (9).csv")

val_df = pd.read_csv(r"C:\Users\USER\Datasets\Text-Deshuffling\val (1).csv")

In [ ]:
#using huggingface's dataset function to load multiple datasets at once 
dataset = load_dataset( 'csv',data_files = {'train':[r"C:\Users\USER\Datasets\Text-Deshuffling\train (8).csv"],
                                         'val':[r"C:\Users\USER\Datasets\Text-Deshuffling\val (1).csv"],
                                         'test':[r"C:\Users\USER\Datasets\Text-Deshuffling\test (9).csv"]})

Using custom data configuration default-53fd3e7085915bfd
Reusing dataset csv (C:\Users\USER\.cache\huggingface\datasets\csv\default-53fd3e7085915bfd\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
new_df= train_df

In [ ]:
dataset

Signature:
dataset.map(
    function: Optional[Callable] = None,
    with_indices: bool = False,
    with_rank: bool = False,
    input_columns: Union[str, List[str], NoneType] = None,
    batched: bool = False,
    batch_size: Optional[int] = 1000,
    drop_last_batch: bool = False,
    remove_columns: Union[str, List[str], NoneType] = None,
    keep_in_memory: bool = False,
    load_from_cache_file: bool = True,
    cache_file_names: Optional[Dict[str, Optional[str]]] = None,
    writer_batch_size: Optional[int] = 1000,
    features: Optional[datasets.features.features.Features] = None,
    disable_nullable: bool = False,
    fn_kwargs: Optional[dict] = None,
    num_proc: Optional[int] = None,
    desc: Optional[str] = None,
) -> 'DatasetDict'
Source:   
    def map(
        self,
        function: Optional[Callable] = None,
        with_indices: bool = False,
        with_rank: bool = False,
        input_columns: Optional[Union[str, List[str]]] = None,
        batched: bool = Fals

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
sample = train_df['label'][10]
text = train_df['text'].astype(str).to_list()
tokens_t = tokenizer(text,truncation = True,max_length = max_text_length,is_split_into_words=True)
tokens_t


{'input_ids': [101, 3591, 2182, 7297, 1010, 15743, 5301, 1012, 7375, 2022, 1996, 2064, 9207, 1037, 1999, 1037, 2433, 9207, 4800, 22172, 6132, 19301, 2686, 1012, 2216, 21374, 11892, 1997, 1996, 2944, 2007, 3522, 2003, 2944, 25582, 1037, 14260, 10949, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
new_df['labels']=tokens_t.input_ids
new_df

ValueError: Length of values (40) does not match length of index (40001)

In [ ]:
tokens = tokenizer.tokenize(sample)
token_id = tokenizer.convert_tokens_to_ids(tokens)
print('sample text: ', sample)
print('tokens : ',tokens)
print('token_id : ',token_id)

sample text:  Properties used were primary, secondary and tertiary structures, accessibility and packing.
tokens :  ['properties', 'used', 'were', 'primary', ',', 'secondary', 'and', 'tertiary', 'structures', ',', 'accessibility', 'and', 'packing', '.']
token_id :  [5144, 2109, 2020, 3078, 1010, 3905, 1998, 13553, 5090, 1010, 23661, 1998, 14743, 1012]


In [ ]:
original_text = tokenizer.decode(token_id)
original_text

'properties used were primary, secondary and tertiary structures, accessibility and packing.'

In [ ]:
def preprocess_function(sample):
    # Getting text and label
    text = sample["text"]
    label = sample["label"]
    # Tokenizing the text and label
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_text_length)
    outputs = tokenizer(label, padding="max_length", truncation=True, max_length=max_label_length)
    sample["input_ids"] = inputs.input_ids
    sample["attention_mask"] = inputs.attention_mask
    sample["decoder_input_ids"] = outputs.input_ids
    sample["decoder_attention_mask"] = outputs.attention_mask
    sample["labels"] = outputs.input_ids
    # The labels are used to calcuate the loss while training, and because we added padding to make all tokens to be of same size,
    # we also need to convert the padding number ( 0 ) to ( -100 ), so that we can tell huggingface that these number can be ignorned while calcuating loss. 
    # Why specifically -100 ? It's simply an arbitrary number, again so that huggingface can ignore this number while calcuating loss
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]

    return sample

In [ ]:
#creating the dataset 
max_text_length = 40 
max_label_length = 40
def preprocess_fn(sample):
    
    text = sample['text']
    label = sample['label']
    inputs = tokenizer(text,padding = 'max_length' , truncation = True,max_length = max_text_length,is_split_into_words=True)
    outputs = tokenizer(label,padding = 'max_length' , truncation = True,max_length = max_label_length,is_split_into_words=True)
    
    sample['input_id'] = inputs.input_ids
    sample['attention_mask'] = inputs.attention_mask
    sample['decoder_input_ids'] = outputs.input_ids
    sample['decoder_attention_mask'] = outputs.attention_mask
    sample['labels'] = outputs.input_ids
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]
    return sample

In [ ]:
tok_dataset=dataset.map(preprocess_function,batch_size=batch,batched=True)

  0%|          | 0/2501 [00:00<?, ?ba/s]

  0%|          | 0/251 [00:00<?, ?ba/s]

  0%|          | 0/625 [00:00<?, ?ba/s]

In [ ]:
tok_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relatio

In [ ]:
model#model architecture

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

In [ ]:
N_epochs = 1
args = Seq2SeqTrainingArguments(
    "Scambled Text",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    num_train_epochs=N_epochs,
    fp16=False, # This will help in increasing the speed of training
)

In [ ]:
trainer = Seq2SeqTrainer(model = model,
                         args = args,
                         train_dataset = tok_dataset['train'],
                         eval_dataset = tok_dataset['val']
                        )

                                            
                         
                         

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: text, id. If text, id are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
C:\Users\USER\.conda\envs\torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40001
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2501
C:\Users\USER\.conda\envs\torch\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train e

Epoch,Training Loss,Validation Loss


In [ ]:
def predictions(test):
    inputs = tokenizer(test['text'],truncation= True,padding = 'max_length',max_length = 'max_text_length')
    
    inputs_id = inputs.inputs_id
    attention_masks = inputs.attention_masks
    
    outputs = model.generate(inputs_id,attention_masks)
    output_str = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    test['prediction'] = output_str
    
    
    

In [ ]:
result = dataset['test'].map(predictions,batch_size = batch,batched=True)